<a href="https://colab.research.google.com/github/Dark-Sied/Intent_Classification/blob/master/Intent_classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intent Classification using Deep Learning

## 1. Loading Data

In [44]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, GRU, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint

Dataset link: https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/train.csv

In [2]:
Intent = "category"
Sentence = "text"

In [3]:
def load_dataset(filename, Sentence, Intent):
  df = pd.read_csv(filename, names = [Sentence, Intent])
  intent = df[Intent]
  unique_intent = list(set(intent))
  sentences = list(df[Sentence])
  
  return (df, intent, unique_intent, sentences)

In [4]:
df, intent, unique_intent, sentences = load_dataset("Dataset.csv", "text", "category")

In [5]:
print(df.head(10))

                                                text      category
0                                               text      category
1                     I am still waiting on my card?  card_arrival
2  What can I do if my card still hasn't arrived ...  card_arrival
3  I have been waiting over a week. Is the card s...  card_arrival
4  Can I track my card while it is in the process...  card_arrival
5  How do I know if I will get my card, or if it ...  card_arrival
6                  When did you send me my new card?  card_arrival
7       Do you have info about the card on delivery?  card_arrival
8  What do I do if I still have not received my n...  card_arrival
9       Does the package with my card have tracking?  card_arrival


In [6]:
import seaborn as sns
import tkinter
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
sns.countplot(x=Intent, data=df)

<AxesSubplot:xlabel='category', ylabel='count'>

In [7]:
print(sentences[:5])

['text', 'I am still waiting on my card?', "What can I do if my card still hasn't arrived after 2 weeks?", 'I have been waiting over a week. Is the card still coming?', 'Can I track my card while it is in the process of delivery?']


In [8]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shiningflash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/shiningflash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 2. Data Cleaning

In [9]:
#define stemmer
stemmer = LancasterStemmer()

In [10]:
def cleaning(sentences):
  words = []
  for s in sentences:
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
    w = word_tokenize(clean)
    words.append([i.lower() for i in w])
    
  return words  

In [11]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  

10004
[['text'], ['i', 'am', 'still', 'waiting', 'on', 'my', 'card']]


## 3. Texts Tokenization

In [12]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [13]:
def max_length(words):
  return(len(max(words, key = len)))

In [14]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 2343 and Maximum length = 84


In [15]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [16]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [17]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [18]:
padded_doc = padding_doc(encoded_doc, max_length)

In [19]:
padded_doc[:5]

array([[1481,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [   1,   50,   64,  208,   30,    2,    6,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,

In [20]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (10004, 84)


In [21]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')

In [22]:
output_tokenizer.word_index

{'top_up_by_cash_or_cheque': 1,
 'card_delivery_estimate': 2,
 'visa_or_mastercard': 3,
 'wrong_amount_of_cash_received': 4,
 'beneficiary_not_allowed': 5,
 'card_payment_wrong_exchange_rate': 6,
 'verify_source_of_funds': 7,
 'top_up_by_card_charge': 8,
 'pin_blocked': 9,
 'automatic_top_up': 10,
 'change_pin': 11,
 'age_limit': 12,
 'edit_personal_details': 13,
 'declined_cash_withdrawal': 14,
 'card_linking': 15,
 'order_physical_card': 16,
 'fiat_currency_support': 17,
 'declined_transfer': 18,
 'topping_up_by_card': 19,
 'top_up_limits': 20,
 'why_verify_identity': 21,
 'declined_card_payment': 22,
 'cancel_transfer': 23,
 'transfer_into_account': 24,
 'wrong_exchange_rate_for_cash_withdrawal': 25,
 'top_up_failed': 26,
 'failed_transfer': 27,
 'card_about_to_expire': 28,
 'request_refund': 29,
 'lost_or_stolen_phone': 30,
 'reverted_card_payment': 31,
 'transaction_charged_twice': 32,
 'pending_top_up': 33,
 'balance_not_updated_after_cheque_or_cash_deposit': 34,
 'verify_top_up'

In [23]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [24]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [25]:
encoded_output.shape

(10004, 1)

In [26]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [27]:
output_one_hot = one_hot(encoded_output)

In [28]:
output_one_hot.shape

(10004, 78)

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)

In [31]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (8003, 84) and train_Y = (8003, 78)
Shape of val_X = (2001, 84) and val_Y = (2001, 78)


## 4. Bidirectional GRU 

In [32]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
  model.add(Bidirectional(GRU(128)))
  model.add(Dense(32, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(78, activation = "softmax"))
  
  return model

In [33]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 84, 128)           299904    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               198144    
_________________________________________________________________
dense (Dense)                (None, 32)                8224      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 78)                2574      
Total params: 508,846
Trainable params: 208,942
Non-trainable params: 299,904
_________________________________________________________________


In [34]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

In [35]:
hist = model.fit(train_X, train_Y,
                 epochs = 100,
                 batch_size = 32,
                 validation_data = (val_X, val_Y),
                 callbacks = [checkpoint])

Epoch 1/100
250/251 [============================>.] - ETA: 0s - loss: 4.3335 - accuracy: 0.0179
Epoch 00001: val_loss improved from inf to 4.27058, saving model to model.h5
251/251 [==============================] - 23s 93ms/step - loss: 4.3334 - accuracy: 0.0179 - val_loss: 4.2706 - val_accuracy: 0.0215
Epoch 2/100
251/251 [==============================] - ETA: 0s - loss: 4.1094 - accuracy: 0.0352
Epoch 00002: val_loss improved from 4.27058 to 3.81736, saving model to model.h5
251/251 [==============================] - 26s 103ms/step - loss: 4.1094 - accuracy: 0.0352 - val_loss: 3.8174 - val_accuracy: 0.0690
Epoch 3/100
251/251 [==============================] - ETA: 0s - loss: 3.7381 - accuracy: 0.0740
Epoch 00003: val_loss improved from 3.81736 to 3.41435, saving model to model.h5
251/251 [==============================] - 27s 106ms/step - loss: 3.7381 - accuracy: 0.0740 - val_loss: 3.4143 - val_accuracy: 0.1224
Epoch 4/100
250/251 [============================>.] - ETA: 0s - loss

Epoch 28/100
251/251 [==============================] - ETA: 0s - loss: 1.6319 - accuracy: 0.4842
Epoch 00028: val_loss did not improve from 1.57346
251/251 [==============================] - 44s 174ms/step - loss: 1.6319 - accuracy: 0.4842 - val_loss: 1.5962 - val_accuracy: 0.5632
Epoch 29/100
251/251 [==============================] - ETA: 0s - loss: 1.6075 - accuracy: 0.4882
Epoch 00029: val_loss did not improve from 1.57346
251/251 [==============================] - 44s 175ms/step - loss: 1.6075 - accuracy: 0.4882 - val_loss: 1.5836 - val_accuracy: 0.5632
Epoch 30/100
251/251 [==============================] - ETA: 0s - loss: 1.5591 - accuracy: 0.4993
Epoch 00030: val_loss improved from 1.57346 to 1.52777, saving model to model.h5
251/251 [==============================] - 44s 174ms/step - loss: 1.5591 - accuracy: 0.4993 - val_loss: 1.5278 - val_accuracy: 0.5782
Epoch 31/100
251/251 [==============================] - ETA: 0s - loss: 1.5530 - accuracy: 0.5071
Epoch 00031: val_loss i

Epoch 57/100
251/251 [==============================] - ETA: 0s - loss: 0.9439 - accuracy: 0.6802
Epoch 00057: val_loss did not improve from 1.44457
251/251 [==============================] - 29s 116ms/step - loss: 0.9439 - accuracy: 0.6802 - val_loss: 1.5952 - val_accuracy: 0.6642
Epoch 58/100
251/251 [==============================] - ETA: 0s - loss: 0.9087 - accuracy: 0.6946
Epoch 00058: val_loss did not improve from 1.44457
251/251 [==============================] - 29s 117ms/step - loss: 0.9087 - accuracy: 0.6946 - val_loss: 1.5913 - val_accuracy: 0.6622
Epoch 59/100
250/251 [============================>.] - ETA: 0s - loss: 0.9004 - accuracy: 0.6955
Epoch 00059: val_loss did not improve from 1.44457
251/251 [==============================] - 29s 115ms/step - loss: 0.9004 - accuracy: 0.6955 - val_loss: 1.6798 - val_accuracy: 0.6532
Epoch 60/100
250/251 [============================>.] - ETA: 0s - loss: 0.8781 - accuracy: 0.6970
Epoch 00060: val_loss did not improve from 1.44457
25

250/251 [============================>.] - ETA: 0s - loss: 0.6293 - accuracy: 0.7812
Epoch 00086: val_loss did not improve from 1.44457
251/251 [==============================] - 23s 90ms/step - loss: 0.6293 - accuracy: 0.7813 - val_loss: 2.0607 - val_accuracy: 0.7016
Epoch 87/100
250/251 [============================>.] - ETA: 0s - loss: 0.6056 - accuracy: 0.7820
Epoch 00087: val_loss did not improve from 1.44457
251/251 [==============================] - 22s 87ms/step - loss: 0.6054 - accuracy: 0.7821 - val_loss: 1.9644 - val_accuracy: 0.6902
Epoch 88/100
250/251 [============================>.] - ETA: 0s - loss: 0.6456 - accuracy: 0.7799
Epoch 00088: val_loss did not improve from 1.44457
251/251 [==============================] - 22s 87ms/step - loss: 0.6462 - accuracy: 0.7798 - val_loss: 2.0080 - val_accuracy: 0.6852
Epoch 89/100
250/251 [============================>.] - ETA: 0s - loss: 0.6215 - accuracy: 0.7843
Epoch 00089: val_loss did not improve from 1.44457
251/251 [=========

## 5. Bidirectional LSTM 

In [45]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
  model.add(Bidirectional(GRU(128)))
  model.add(Dense(32, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(78, activation = "softmax"))
  
  return model

model_lstm = create_model(vocab_size, max_length)

model_lstm.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model_lstm.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 84, 128)           299904    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               198144    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 78)                2574      
Total params: 508,846
Trainable params: 208,942
Non-trainable params: 299,904
_________________________________________________________________


In [47]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

hist = model_lstm.fit(train_X, train_Y,
                 epochs = 100,
                 batch_size = 32,
                 validation_data = (val_X, val_Y),
                 callbacks = [checkpoint])

Epoch 1/100
250/251 [============================>.] - ETA: 0s - loss: 4.3337 - accuracy: 0.0188
Epoch 00001: val_loss improved from inf to 4.27265, saving model to model.h5
251/251 [==============================] - 22s 86ms/step - loss: 4.3337 - accuracy: 0.0187 - val_loss: 4.2727 - val_accuracy: 0.0240
Epoch 2/100
251/251 [==============================] - ETA: 0s - loss: 4.1330 - accuracy: 0.0307
Epoch 00002: val_loss improved from 4.27265 to 3.94408, saving model to model.h5
251/251 [==============================] - 30s 118ms/step - loss: 4.1330 - accuracy: 0.0307 - val_loss: 3.9441 - val_accuracy: 0.0460
Epoch 3/100
250/251 [============================>.] - ETA: 0s - loss: 3.8935 - accuracy: 0.0562
Epoch 00003: val_loss improved from 3.94408 to 3.64672, saving model to model.h5
251/251 [==============================] - 30s 120ms/step - loss: 3.8934 - accuracy: 0.0564 - val_loss: 3.6467 - val_accuracy: 0.0820
Epoch 4/100
250/251 [============================>.] - ETA: 0s - loss

Epoch 28/100
251/251 [==============================] - ETA: 0s - loss: 1.5961 - accuracy: 0.4863
Epoch 00028: val_loss improved from 1.53518 to 1.52079, saving model to model.h5
251/251 [==============================] - 24s 96ms/step - loss: 1.5961 - accuracy: 0.4863 - val_loss: 1.5208 - val_accuracy: 0.5637
Epoch 29/100
250/251 [============================>.] - ETA: 0s - loss: 1.5584 - accuracy: 0.4946
Epoch 00029: val_loss improved from 1.52079 to 1.51989, saving model to model.h5
251/251 [==============================] - 21s 84ms/step - loss: 1.5580 - accuracy: 0.4947 - val_loss: 1.5199 - val_accuracy: 0.5597
Epoch 30/100
250/251 [============================>.] - ETA: 0s - loss: 1.5395 - accuracy: 0.5059
Epoch 00030: val_loss improved from 1.51989 to 1.47681, saving model to model.h5
251/251 [==============================] - 22s 88ms/step - loss: 1.5393 - accuracy: 0.5059 - val_loss: 1.4768 - val_accuracy: 0.5772
Epoch 31/100
250/251 [============================>.] - ETA: 0s 

250/251 [============================>.] - ETA: 0s - loss: 0.9419 - accuracy: 0.6755
Epoch 00056: val_loss did not improve from 1.39536
251/251 [==============================] - 21s 85ms/step - loss: 0.9421 - accuracy: 0.6755 - val_loss: 1.5368 - val_accuracy: 0.6372
Epoch 57/100
250/251 [============================>.] - ETA: 0s - loss: 0.9355 - accuracy: 0.6697
Epoch 00057: val_loss did not improve from 1.39536
251/251 [==============================] - 21s 82ms/step - loss: 0.9354 - accuracy: 0.6697 - val_loss: 1.5792 - val_accuracy: 0.6407
Epoch 58/100
251/251 [==============================] - ETA: 0s - loss: 0.9021 - accuracy: 0.6909
Epoch 00058: val_loss did not improve from 1.39536
251/251 [==============================] - 24s 97ms/step - loss: 0.9021 - accuracy: 0.6909 - val_loss: 1.5153 - val_accuracy: 0.6442
Epoch 59/100
250/251 [============================>.] - ETA: 0s - loss: 0.8918 - accuracy: 0.6971
Epoch 00059: val_loss did not improve from 1.39536
251/251 [=========

250/251 [============================>.] - ETA: 0s - loss: 0.6162 - accuracy: 0.7835
Epoch 00085: val_loss did not improve from 1.39536
251/251 [==============================] - 24s 94ms/step - loss: 0.6160 - accuracy: 0.7836 - val_loss: 1.9181 - val_accuracy: 0.6812
Epoch 86/100
250/251 [============================>.] - ETA: 0s - loss: 0.6228 - accuracy: 0.7818
Epoch 00086: val_loss did not improve from 1.39536
251/251 [==============================] - 23s 92ms/step - loss: 0.6228 - accuracy: 0.7817 - val_loss: 1.9226 - val_accuracy: 0.6847
Epoch 87/100
251/251 [==============================] - ETA: 0s - loss: 0.6602 - accuracy: 0.7672
Epoch 00087: val_loss did not improve from 1.39536
251/251 [==============================] - 20s 82ms/step - loss: 0.6602 - accuracy: 0.7672 - val_loss: 1.9660 - val_accuracy: 0.6807
Epoch 88/100
250/251 [============================>.] - ETA: 0s - loss: 0.6757 - accuracy: 0.7626
Epoch 00088: val_loss did not improve from 1.39536
251/251 [=========

In [48]:
 model_lstm = load_model("model.h5")

In [49]:
def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)

  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
  x = padding_doc(test_ls, max_length)

  pred = model_lstm.predict(x)
  
  return pred

In [50]:
def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))
  
  return classes[0]

In [51]:
text = "I am still waiting on my card?"
pred = predictions(text)
result = get_final_output(pred, unique_intent)
print('\nans: {}\n'.format(result))

['i', 'am', 'still', 'waiting', 'on', 'my', 'card']
reverted_card_payment? has confidence = 0.18450873
card_arrival has confidence = 0.15817639
card_linking has confidence = 0.12097399
card_delivery_estimate has confidence = 0.1064741
compromised_card has confidence = 0.10200037
declined_card_payment has confidence = 0.05154416
lost_or_stolen_card has confidence = 0.049066834
transaction_charged_twice has confidence = 0.047501627
card_payment_not_recognised has confidence = 0.036972415
cash_withdrawal_not_recognised has confidence = 0.031303987
request_refund has confidence = 0.031057216
card_not_working has confidence = 0.021079693
pending_card_payment has confidence = 0.017942613
contactless_not_working has confidence = 0.008497491
pending_top_up has confidence = 0.008054751
Refund_not_showing_up has confidence = 0.004341421
balance_not_updated_after_bank_transfer has confidence = 0.004288543
transfer_not_received_by_recipient has confidence = 0.0033927432
balance_not_updated_after_c

In [52]:
text = "What are you exchange rates?"
pred = predictions(text)
result = get_final_output(pred, unique_intent)
print('\nans: {}\n'.format(result))

['what', 'are', 'you', 'exchange', 'rates']
exchange_rate has confidence = 0.9521722
fiat_currency_support has confidence = 0.018024959
exchange_via_app has confidence = 0.017376026
exchange_charge has confidence = 0.008700637
supported_cards_and_currencies has confidence = 0.0014381895
card_payment_wrong_exchange_rate has confidence = 0.001223994
wrong_exchange_rate_for_cash_withdrawal has confidence = 0.0008252348
apple_pay_or_google_pay has confidence = 0.00023869329
receiving_money has confidence = 2.8229067e-08
top_up_by_card_charge has confidence = 9.1513563e-10
direct_debit_payment_not_recognised has confidence = 5.412927e-10
card_acceptance has confidence = 3.9122233e-10
top_up_by_bank_transfer_charge has confidence = 2.2138892e-10
transfer_fee_charged has confidence = 1.8366568e-10
beneficiary_not_allowed has confidence = 1.04932375e-10
declined_card_payment has confidence = 4.518701e-11
reverted_card_payment? has confidence = 1.4193713e-11
card_payment_not_recognised has conf

In [53]:
text = "Which countries are represented?"
pred = predictions(text)
result = get_final_output(pred, unique_intent)
print('\nans: {}\n'.format(result))

['which', 'countries', 'are', 'represented']
country_support has confidence = 0.40900862
fiat_currency_support has confidence = 0.21668743
card_acceptance has confidence = 0.083111525
supported_cards_and_currencies has confidence = 0.06816458
getting_spare_card has confidence = 0.053985287
atm_support has confidence = 0.04560661
order_physical_card has confidence = 0.04163523
exchange_via_app has confidence = 0.019778987
card_about_to_expire has confidence = 0.01835203
age_limit has confidence = 0.009834507
receiving_money has confidence = 0.008849052
card_delivery_estimate has confidence = 0.004631613
top_up_by_card_charge has confidence = 0.0040301057
compromised_card has confidence = 0.0023763773
card_not_working has confidence = 0.001980662
card_payment_fee_charged has confidence = 0.0015887762
activate_my_card has confidence = 0.0014576323
disposable_card_limits has confidence = 0.001454433
exchange_rate has confidence = 0.0011822341
lost_or_stolen_card has confidence = 0.00101401